In [1]:
!pwd
# Import python modules
import pandas as pd
import pprint
import json
import matplotlib.pyplot as plt
import seaborn as sns
#import deepchem as dc
#print(dc.__version__)

# We temporarily disable warnings for demonstration.
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/usr/WS2/atom/titli/MoDaC/Codes


In [2]:
# Import AMPL modules
import atomsci.ddm.pipeline
#from atomsci.ddm.pipeline import predict_from_model as pfm  # only need this

In [ ]:
print("Hello World")

In [ ]:
!pwd 
data_dir = 'NCI_DOE_Archive/ATOM/Safety_Screen_Targets/'
%ls $data_dir
len_data_dir = len(data_dir.split("/"))-1

# output dir
ampl140_dir = data_dir + 'output_ampl-1.4.0/'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tempfile,tarfile,os,stat
from sklearn.metrics import r2_score,accuracy_score, mean_squared_error, roc_auc_score, adjusted_rand_score, f1_score 
from sklearn.metrics import roc_curve, precision_recall_curve

def test_model(model_path, data_path): # only model/data name is passed here. They have to be in the same location
  gene = model_path.split("_")[0] # gene name
  print("Gene = %s" % gene)

  # extract test data
  reload_dir = tempfile.mkdtemp()
  data_fp = tarfile.open(data_dir + data_path, mode='r:gz') # open the tarfile, extractall and close
  data_fp.extractall(path = reload_dir)
  data_fp.close()
  # view contents of the extracted DATA tar file from temp folder
  test_data = [l for l in os.listdir(reload_dir) if 'testset' in l] # [dtc, chembl, excape, union]
  print(len(test_data),test_data)dir(reload_dir) if 'trainset' in l] # [dtc, chembl, excape, union]
  print(len(test_data),test_data) 
  train_data = [l for l in os.list 

  # run prediction on each model for each gene sample with four data sources 
  def run_prediction(data, dtype):
      models_df = pd.DataFrame(columns=['gene', 'data_source', 'dist_metric', 'MSE', 'R2', 'ARI'])
      for i in data:
        print("\n", i)
        df = pd.read_csv(os.path.join(reload_dir, i))
        df.rename(columns = {"PIC50": "pIC50"}, inplace = True)
        df.rename(columns = {"pXC50": "pIC50"}, inplace = True) # X - I or E
        print(df.columns)

        dist_metric  = ['cityblock', 'cosine', 'euclidean', 'jaccard', 'manhattan']
        for dist in dist_metric:
          #pred_df = pfm.predict_from_model_file(data_dir+model_path, df, id_col='compound_id', smiles_col='base_rdkit_smiles',\
                          #response_col='pIC50', conc_col=None, is_featurized=False, dont_standardize=False, AD_method=None, k=5, \
                          #dist_metric="euclidean", external_training_data=None)

          pred_df = pfm.predict_from_model_file(ampl140_dir + model_path, df, \
                                                id_col='compound_id', smiles_col='base_rdkit_smiles', response_col='pIC50', \
                                                dist_metric=dist)
          print(gene, i, dist, pred_df.columns)

          r2 = np.round(r2_score(pred_df['pIC50_actual'], pred_df['pIC50_pred']), 2)
          print("r2_score = ", r2)
          mse =  np.round(mean_squared_error(pred_df['pIC50_actual'], pred_df['pIC50_pred']), 2)
          print("mean_squared_error (MSE) = ", mse)
          ari =  np.round(adjusted_rand_score(pred_df['pIC50_actual'], pred_df['pIC50_pred']), 2)
          print("adjusted_rand_score (ARI) = ", ari)

          models_df.loc[len(models_df.index)] = [gene, i, dist, mse, r2, ari]

          # plot
          '''plt.figure(figsize=(5,5))
          actual_value = pred_df['pIC50_actual']
          predicted_value = pred_df['pIC50_pred']
          plt.scatter(actual_value, predicted_value, c='crimson')
          p1 = max(max(predicted_value), max(actual_value))
          p2 = min(min(predicted_value), min(actual_value))
          plt.plot([p1, p2], [p1, p2], 'b-')
          plt.xlabel('Actual')
          plt.ylabel('Predicted')
          plt.title(i+'_'+ dist)
          plt.show()'''
      print(model_path, "\n", data_path, "\n", models_df)
      # save comparion results for each gene in a dataframe 
      models_df.to_csv(ampl140_dir + gene + '_' + dtype + '.csv', encoding = 'utf-8-sig')
 
  run_prediction(test_data, "test")
  #run_prediction(train_data, "train")

In [ ]:
#genes = ['CHRM1', 'CHRM2', 'CHRM3', 'CYP2C9', 'CYP2D6', 'CYP3A4', 'HRH1', 'KCNH2', 'PIK3CG']
genes = ['CYP3A4'] 
print(len(genes),genes)
for gene in genes:
  # data filepath
  data_path = gene.upper()+'-ampl-1.1.0.tar.gz'
    
  #model file path (I have renamed the code generated model name as per MoDaC naming convention)
  if gene in ['CHRM1','CHRM2', 'CHRM3','HRH1', 'KCNH2']: 
    model_path = gene.lower() + "_union_trainset_graphconv_ampl-1.4.0.tar.gz";
  if gene in ['CYP2C9', 'CYP2D6', 'CYP3A4']:
    model_path = gene.lower() + "_union_trainset_mordred_ampl-1.4.0.tar.gz";
  #model_path = 'cyp3a4_union_trainset_base_smiles_model_f69c686c-0d16-4395-9526-8c2d96c60979.tar.gz'
  print(model_path, data_path)
  test_model(model_path, data_path)

print("CODE END")

In [ ]:
#!ls -ltr NCI_DOE_Archive/ATOM/Safety_Screen_Targets/output_ampl-1.4.0/

In [ ]:
import sys 
print(sys.version)

In [5]:
data_dir = './../NCI_DOE_Archive/ATOM/Safety_Screen_Targets/output_ampl-1.4.0_FRCE/'
#/usr/WS2/atom/titli/MoDaC/NCI_DOE_Archive/ATOM/Safety_Screen_Targets/output_ampl-1.4.0_FRCE
!ls $data_dir

chrm1_union_trainset_base_smiles
chrm1_union_trainset_base_smiles_model_1bc99b84-d648-4c05-a9e0-a207bc5f95d0.tar.gz
chrm2_union_trainset_base_smiles
chrm2_union_trainset_base_smiles_model_f5b07cfc-e997-4432-b8c4-efe6167dad65.tar.gz
chrm3_union_trainset_base_smiles
chrm3_union_trainset_base_smiles_model_745e4fb3-d3a8-4ed7-acd3-fcaac0dffac9.tar.gz
cyp2c9_union_trainset_base_smiles
cyp2c9_union_trainset_base_smiles_model_1effb9ec-f9eb-4e8e-a6ac-523349ce061d.tar.gz
cyp2d6_union_trainset_base_smiles
cyp2d6_union_trainset_base_smiles_model_4109d3b2-062e-494a-b69b-d2b2364e3461.tar.gz
cyp3a4_union_trainset_base_smiles
cyp3a4_union_trainset_base_smiles_model_edd9e3d3-df64-443f-be50-54e807d78d9f.tar.gz
hrh1_union_trainset_base_smiles
hrh1_union_trainset_base_smiles_model_ebba0d8c-0111-443b-9654-bd203f07a42f.tar.gz
kcnh2_union_trainset_base_smiles
kcnh2_union_trainset_base_smiles_model_32d3656d-b731-482a-9135-e0a70235f1a0.tar.gz


In [25]:
import os, tempfile, tarfile
genes = ['CHRM1', 'CHRM2', 'CHRM3', 'CYP2C9', 'CYP2D6', 'CYP3A4', 'HRH1', 'KCNH2', 'PIK3CG']
for gene in genes:
    model_path = [file for file in os.listdir(data_dir) if file.startswith(gene.lower()+'_union_trainset_base_smiles_model_')][0]
    #print(model_path)
    # extract test data
    model_extracted_dir = data_dir + model_path.replace('.tar.gz','') + '/'
    if not os.path.exists(model_extracted_dir):os.makedirs(model_extracted_dir)
    with tarfile.open(data_dir + model_path , mode='r:gz') as tar:tar.extractall(path = model_extracted_dir) 
    with open(model_extracted_dir+'model_metadata.json') as json_file:
        data = json.load(json_file)
        ampl_version = data['model_parameters']['ampl_version']
        featurizer = data['model_parameters']['featurizer']
        descriptor='None'
        if (featurizer == 'computed_descriptors'):
            descriptor = data['descriptor_specific']['descriptor_type']
        print(gene,featurizer, descriptor,ampl_version)
        

CHRM1 graphconv None 1.4.0
CHRM2 graphconv None 1.4.0
CHRM3 graphconv None 1.4.0
CYP2C9 computed_descriptors mordred_filtered 1.4.0
CYP2D6 computed_descriptors mordred_filtered 1.4.0
CYP3A4 computed_descriptors mordred_filtered 1.4.0
HRH1 graphconv None 1.4.0
KCNH2 graphconv None 1.4.0
PIK3CG computed_descriptors mordred_filtered 1.4.0
